In [19]:
import argparse
import json
import logging
import os
import sys

import pandas as pd

from utils.utils import separate_dps, file_tqdm, separate_lrs
from utils.tree_utils import *
from dataset import *

In [49]:
data_dir = "../data"
journalist = "nighatdad"
out_dir = "../data/"
n_ctx = 2000
max_width = 16
max_depth = 10
td = True
local_relation = True

num_dps = 0
num_classes = 3

journal_sort = pd.read_csv(os.path.join(data_dir, f'{journalist}/{journalist}_conv_labels.csv'))
num_sequences = len(set(journal_sort['conversation_id']))

journal_batch = journal_sort[["type", "possibly_sensitive", "lang", "reply_settings",
                                "retweet_count", "reply_count", "like_count", "quote_count", "impression_count",
                                "mentions", "urls", "labels"]]
ids = []
for item in list(journal_sort['conversation_id']):
    if item not in ids:
        ids.append(item)
id_pair = {}
for idx in ids:
    id_pair[idx] = create_conversation_list(journal_sort[journal_sort['conversation_id']==idx], idx)


In [50]:
len(ids)

166

In [51]:

if local_relation:
    print("Save Relation between Child and Father")
    with open(os.path.join(out_dir, f'{journalist}/{journalist}_local_path.txt'), "w") as fout:
        for k in id_pair.keys():
            tree_root = build_tree(id_pair[k][0])
            
            tree_root.create_local_relation()
            node_list = tree_root.dfs()
            name_list = []
            for item in node_list:
                name_list.append(item.name)

            id_map, id_re = id_mapping(name_list)
            local_relations = TreeNode.extract_data(node_list,f=lambda node: node.local_relation)
            """lrs = separate_lrs(local_relations, id_map, id_re, n_ctx)

            for lr, extended in lrs:
                if extended != 0:
                    break
                if len(lr) - extended > 1:
                    json.dump(lr, fp=fout)  # each line is the json of a list [dict,dict,...]
                    num_dps += 1
                    fout.write("\n")"""
            json.dump(local_relations, fp=fout)
            fout.write("\n")




Save Relation between Child and Father


In [52]:
# sallykohn
# journal_sort = journal_sort.drop(10).reset_index(drop=True)
# journal_sort.to_csv(os.path.join(data_dir, f'{journalist}/{journalist}_conv_labels.csv'))

In [53]:
if td:
    print("Save Child Paths and Father Paths")

num_dps = 0
with open(os.path.join(out_dir, f'{journalist}/{journalist}_global_path.txt'), "w") as fout:
    for k in id_pair.keys():
        tree_root = build_tree(id_pair[k][0])
        #print(tree_root.name)
        tree_root.create_global_relation()
        node_list = tree_root.dfs()
        
        root_paths = TreeNode.extract_data(node_list,f=lambda node: clamp_and_slice_ids(
                node.global_relation, max_width=-1, max_depth=-1))
        """asts = separate_dps(root_paths, n_ctx)

        for lr, extended in asts:
            if extended != 0:
                break
            if len(lr) - extended > 1:
                json.dump(lr, fp=fout)  # each line is the json of a list [dict,dict,...]
                num_dps += 1
                fout.write("\n")"""
        json.dump(root_paths, fp=fout)
        fout.write("\n")
        

Save Child Paths and Father Paths


In [7]:
temp = []
for k in id_pair.keys():
    tree_root = build_tree(id_pair[k][0])
    #print(tree_root.name)
    tree_root.create_global_relation()
    node_list = tree_root.dfs()
    
    root_paths = TreeNode.extract_data(node_list,f=lambda node: clamp_and_slice_ids(
            node.global_relation, max_width=-1, max_depth=-1))
    temp.append(root_paths)

In [8]:
temp[0]

[{526: {526: [[0, 0, 0]]}}, {527: {527: [[0, 0, 0], [1, 0, 0]]}}]

In [12]:
split = [60, 120]
data_dir = out_dir
val = 0
journal_sort = pd.read_csv((os.path.join(data_dir, f'{journalist}/{journalist}_conv_labels.csv')))
ids = list(set(journal_sort['conversation_id']))
dp = []
rel = []
id_pair = {}
id_conv = {}
for idx in ids:
    id_pair[idx], id_conv[idx] = create_conversation_list(journal_sort[journal_sort['conversation_id']==idx], idx)
id_data, data, label = create_data(journal_sort, ids)
prob = pkl.load(open(os.path.join(data_dir, f'{journalist}/{journalist}_edgeprob.pkl'), 'rb'))


In [13]:
dp = []
rel = []
with open(os.path.join(data_dir, f'{journalist}/{journalist}_global_path.txt'), "r") as f:
    for line in tqdm(f, total=get_number_of_lines(f)):
        dp.append(json.loads(line.strip()))

with open(os.path.join(data_dir, f'{journalist}/{journalist}_local_path.txt'), "r") as f:
    for line in tqdm(f, total=get_number_of_lines(f)):
        rel.append(json.loads(line.strip()))

100%|██████████| 419/419 [00:00<00:00, 3179.51it/s]


In [14]:
global_input = convert_global(dp, id_data)